In [1]:
!pip install yfinance


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

 Question 1: Generate 1500 data points for the variables x, y, z with the Python (or using another language of your choice) commands.

Do a Principal Components Analysis (PCA) on the sample of three-dimensional observations, and write down the three components and the corresponding principal values. (If they are not vectors of length 3, you are doing something wrong.) Check that the pointwise products of the components are zero (up to machine precision). Write down the dot (i.e. pointwise) product of the first and third components, and compare that with the dot product of xdata and zdata. Show your steps and append your code to the assignment.


In [6]:
# Generate 1500 data points for x, y, and z
mean = np.array([2, 3, 0])
matrix_m = np.array([[10, 7, 5], [7, 6, 4], [5, 4, 3]])
sample = np.random.multivariate_normal(mean, matrix_m, 1500).T

# Separate the data into xdata, ydata, and zdata
x_data = sample[0, :]
y_data = sample[1, :]
z_data = sample[2, :]

#combine the data
xyz_data = np.vstack((x_data, y_data, z_data)).T